 # Selecting a suitable Location for Japanese restaurant in New York

## Import Libraries

In this section we import the libraries that will be required to process the data.

The first library is Pandas.
Pandas is an open source, BSD-licensed library, providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json

!conda install -c anaconda beautifulsoup4 --yes
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         156 KB  anaconda

The following packages will be UPDATED:

    certifi: 2019.11.28-py36_0 conda-forge --> 2019.11.28-py36_0 anaconda
    openssl: 1.1.1d-h516909a_0 conda-forge --> 1.1.1-h7b6447c_0  anaconda

## Download and Explore Dataset


Download and Explore Dataset
Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

Luckily, this dataset exists for free on the web. Feel free to try to find this dataset on your own, but here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

For your convenience, I downloaded the files and placed it on the server, so you can simply run a wget command and access the data. So let's go ahead and do that.

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


#### Tranform the data into a *pandas* dataframe

In [3]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [4]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


#### Use geopy library to get the latitude and longitude values of New York City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [5]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Create a map of New York with neighborhoods superimposed on top.

In [6]:
br_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
br_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [7]:
import folium
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(br_data['Latitude'], br_data['Longitude'], br_data['Borough'], br_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

## Foursquare venues


In [8]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [9]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'ZMHWBS0SR12Z3YDYVHJVTZPRK3U1ZP3I2TYQAJ5CU3JUHMB5'
CLIENT_SECRET = 'H3TT0XT3P5TIAFCV1Y2UUVLF42N44DICNKLUELK34H2TKLFR'
VERSION = '20181020'

In [10]:
#https://developer.foursquare.com/docs/resources/categories
#Japanese = 4bf58dd8d48988d111941735
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
newyork_venues_jr = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d111941735')
newyork_venues_jr.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Inaka,40.625141,-74.030418,Sushi Restaurant
1,Bay Ridge,40.625801,-74.030621,Arirang Hibachi Steakhouse,40.621168,-74.029299,Japanese Restaurant
2,Bay Ridge,40.625801,-74.030621,Omiya,40.622329,-74.028576,Sushi Restaurant
3,Bay Ridge,40.625801,-74.030621,Chopstix Restaurant,40.622670,-74.025465,Asian Restaurant
4,Bay Ridge,40.625801,-74.030621,Tairyo Japanese Fusion,40.631792,-74.027477,Japanese Restaurant


In [11]:
newyork_venues_jr.shape

(817, 7)

In [12]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [13]:
map_newyork_jr = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_jr, 'red', map_newyork_jr)

map_newyork_jr

In [14]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [15]:
brooklyn_grouped = newyork_venues_jr.groupby('Neighborhood').count()
brooklyn_grouped
#print('There are {} uniques categories.'.format(len(newyork_venues_jr['Venue Category'].unique())))

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,23,23,23,23,23,23
Bay Ridge,27,27,27,27,27,27
Bedford Stuyvesant,3,3,3,3,3,3
Bensonhurst,27,27,27,27,27,27
Bergen Beach,3,3,3,3,3,3
Boerum Hill,32,32,32,32,32,32
Borough Park,4,4,4,4,4,4
Brighton Beach,7,7,7,7,7,7
Broadway Junction,2,2,2,2,2,2


## 3. Analyze Each Neighborhood

In [16]:
# one hot encoding
brooklyn_onehot = pd.get_dummies(newyork_venues_jr[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
brooklyn_onehot['Neighborhood'] = newyork_venues_jr['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]

brooklyn_onehot.head()

,Neighborhood,Asian Restaurant,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Dumpling Restaurant,Fish Market,Food Stand,Japanese Restaurant,Kosher Restaurant,Poke Place,Ramen Restaurant,Shabu-Shabu Restaurant,Snack Place,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Udon Restaurant,Vietnamese Restaurant
0,Bay Ridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,Bay Ridge,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,Bay Ridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,Bay Ridge,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Bay Ridge,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [17]:
brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
brooklyn_grouped

,Neighborhood,Asian Restaurant,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Dumpling Restaurant,Fish Market,Food Stand,Japanese Restaurant,Kosher Restaurant,Poke Place,Ramen Restaurant,Shabu-Shabu Restaurant,Snack Place,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Udon Restaurant,Vietnamese Restaurant
0,Bath Beach,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.478261,0.000000,0.000000,0.000000,0.000000,0.000000,0.478261,0.000000,0.000000,0.000000,0.000000
1,Bay Ridge,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.592593,0.000000,0.000000,0.037037,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000
2,Bedford Stuyvesant,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000
3,Bensonhurst,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.481481,0.000000,0.000000,0.037037,0.037037,0.000000,0.407407,0.000000,0.037037,0.000000,0.000000
4,Bergen Beach,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,0.000000,0.000000,0.000000,0.000000
5,Boerum Hill,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.406250,0.000000,0.031250,0.125000,0.000000,0.000000,0.406250,0.000000,0.000000,0.000000,0.000000
6,Borough Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.250000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000
7,Brighton Beach,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
8,Broadway Junction,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Brooklyn Heights,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.400000,0.000000,0.033333,0.166667,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000


In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Japanese Restaurant,Sushi Restaurant,BBQ Joint,Food Stand,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Dumpling Restaurant
1,Bay Ridge,Japanese Restaurant,Sushi Restaurant,Ramen Restaurant,Asian Restaurant,Tapas Restaurant,Dumpling Restaurant,BBQ Joint,Bubble Tea Shop,Burger Joint,Café
2,Bedford Stuyvesant,Japanese Restaurant,Tapas Restaurant,Ramen Restaurant,Fish Market,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar
3,Bensonhurst,Japanese Restaurant,Sushi Restaurant,Thai Restaurant,Shabu-Shabu Restaurant,Ramen Restaurant,Dumpling Restaurant,BBQ Joint,Bubble Tea Shop,Burger Joint,Café
4,Bergen Beach,Sushi Restaurant,Japanese Restaurant,Food Stand,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Dumpling Restaurant


Cluster Neighborhoods


In [20]:
# set number of clusters
kclusters = 5

brooklyn_grouped_clustering = brooklyn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 2, 0, 4, 0, 0, 4, 1, 0], dtype=int32)

In [21]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

brooklyn_merged = br_data
brooklyn_merged = brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

brooklyn_merged.head() 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,3.0,Japanese Restaurant,Sushi Restaurant,Ramen Restaurant,Asian Restaurant,Tapas Restaurant,Dumpling Restaurant,BBQ Joint,Bubble Tea Shop,Burger Joint,Café
1,Brooklyn,Bensonhurst,40.611009,-73.995180,0.0,Japanese Restaurant,Sushi Restaurant,Thai Restaurant,Shabu-Shabu Restaurant,Ramen Restaurant,Dumpling Restaurant,BBQ Joint,Bubble Tea Shop,Burger Joint,Café
2,Brooklyn,Sunset Park,40.645103,-74.010316,3.0,Japanese Restaurant,Sushi Restaurant,Ramen Restaurant,Asian Restaurant,Tapas Restaurant,Dumpling Restaurant,BBQ Joint,Bubble Tea Shop,Burger Joint,Café
3,Brooklyn,Greenpoint,40.730201,-73.954241,4.0,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Café,Fish Market,BBQ Joint,Bubble Tea Shop,Burger Joint,Chinese Restaurant,Cocktail Bar
4,Brooklyn,Gravesend,40.595260,-73.973471,0.0,Japanese Restaurant,Sushi Restaurant,Food Stand,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Dumpling Restaurant


In [22]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(brooklyn_merged['Latitude'], brooklyn_merged['Longitude'], brooklyn_merged['Neighborhood'], brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [23]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 0, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Bensonhurst,Japanese Restaurant,Sushi Restaurant,Thai Restaurant,Shabu-Shabu Restaurant,Ramen Restaurant,Dumpling Restaurant,BBQ Joint,Bubble Tea Shop,Burger Joint,Café
4,Gravesend,Japanese Restaurant,Sushi Restaurant,Food Stand,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Dumpling Restaurant
9,Crown Heights,Japanese Restaurant,Sushi Restaurant,Burger Joint,Chinese Restaurant,Vietnamese Restaurant,Snack Place,Shabu-Shabu Restaurant,Ramen Restaurant,Poke Place,Kosher Restaurant
13,Prospect Heights,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Asian Restaurant,Chinese Restaurant,Cocktail Bar,Tapas Restaurant,Dumpling Restaurant,BBQ Joint,Bubble Tea Shop
15,Williamsburg,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Bubble Tea Shop,Udon Restaurant,Asian Restaurant,Shabu-Shabu Restaurant,Snack Place,Poke Place,Kosher Restaurant
16,Bushwick,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Ramen Restaurant,Tapas Restaurant,Dumpling Restaurant,BBQ Joint,Bubble Tea Shop,Burger Joint,Café
18,Brooklyn Heights,Japanese Restaurant,Sushi Restaurant,Ramen Restaurant,Food Stand,Poke Place,Asian Restaurant,Shabu-Shabu Restaurant,Snack Place,Tapas Restaurant,Kosher Restaurant
19,Cobble Hill,Sushi Restaurant,Japanese Restaurant,Snack Place,Ramen Restaurant,Poke Place,Cocktail Bar,Dumpling Restaurant,BBQ Joint,Bubble Tea Shop,Burger Joint
20,Carroll Gardens,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Snack Place,Cocktail Bar,Fish Market,BBQ Joint,Bubble Tea Shop,Burger Joint,Café
22,Gowanus,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Fish Market,Snack Place,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant


In [24]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 1, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Brownsville,Japanese Restaurant,Food Stand,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Dumpling Restaurant,Fish Market
26,East New York,Japanese Restaurant,Food Stand,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Dumpling Restaurant,Fish Market
30,Mill Island,Japanese Restaurant,Food Stand,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Dumpling Restaurant,Fish Market
43,Ocean Hill,Japanese Restaurant,Food Stand,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Dumpling Restaurant,Fish Market
44,City Line,Japanese Restaurant,Food Stand,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Dumpling Restaurant,Fish Market
56,Rugby,Japanese Restaurant,Food Stand,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Dumpling Restaurant,Fish Market
57,Remsen Village,Japanese Restaurant,Food Stand,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Dumpling Restaurant,Fish Market
59,Paerdegat Basin,Japanese Restaurant,Food Stand,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Dumpling Restaurant,Fish Market
64,Broadway Junction,Japanese Restaurant,Food Stand,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Dumpling Restaurant,Fish Market


In [25]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 2, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Flatbush,Japanese Restaurant,Ramen Restaurant,Food Stand,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Dumpling Restaurant
17,Bedford Stuyvesant,Japanese Restaurant,Tapas Restaurant,Ramen Restaurant,Fish Market,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar
63,Weeksville,Tapas Restaurant,Ramen Restaurant,Vietnamese Restaurant,Fish Market,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar


In [26]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 3, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,Japanese Restaurant,Sushi Restaurant,Ramen Restaurant,Asian Restaurant,Tapas Restaurant,Dumpling Restaurant,BBQ Joint,Bubble Tea Shop,Burger Joint,Café
2,Sunset Park,Japanese Restaurant,Sushi Restaurant,Ramen Restaurant,Asian Restaurant,Tapas Restaurant,Dumpling Restaurant,BBQ Joint,Bubble Tea Shop,Burger Joint,Café
35,Dyker Heights,Japanese Restaurant,Sushi Restaurant,Dumpling Restaurant,Ramen Restaurant,Asian Restaurant,Snack Place,Shabu-Shabu Restaurant,Tapas Restaurant,Poke Place,Kosher Restaurant
37,Marine Park,Japanese Restaurant,Sushi Restaurant,Food Stand,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Dumpling Restaurant
47,Prospect Park South,Japanese Restaurant,Sushi Restaurant,Ramen Restaurant,Asian Restaurant,Tapas Restaurant,Dumpling Restaurant,BBQ Joint,Bubble Tea Shop,Burger Joint,Café
48,Georgetown,Japanese Restaurant,Sushi Restaurant,Chinese Restaurant,Food Stand,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Cocktail Bar,Dumpling Restaurant
53,Fort Hamilton,Japanese Restaurant,Sushi Restaurant,Food Stand,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Dumpling Restaurant
54,Ditmas Park,Japanese Restaurant,Sushi Restaurant,Ramen Restaurant,Asian Restaurant,Tapas Restaurant,Dumpling Restaurant,BBQ Joint,Bubble Tea Shop,Burger Joint,Café
60,Mill Basin,Japanese Restaurant,Sushi Restaurant,Chinese Restaurant,Food Stand,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Cocktail Bar,Dumpling Restaurant
69,Erasmus,Japanese Restaurant,Asian Restaurant,Thai Restaurant,Fish Market,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar


In [27]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 4, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Greenpoint,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Café,Fish Market,BBQ Joint,Bubble Tea Shop,Burger Joint,Chinese Restaurant,Cocktail Bar
5,Brighton Beach,Sushi Restaurant,Vietnamese Restaurant,Food Stand,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Dumpling Restaurant
6,Sheepshead Bay,Sushi Restaurant,Japanese Restaurant,Poke Place,Fish Market,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar
7,Manhattan Terrace,Sushi Restaurant,Japanese Restaurant,Café,Kosher Restaurant,Fish Market,BBQ Joint,Bubble Tea Shop,Burger Joint,Chinese Restaurant,Cocktail Bar
11,Kensington,Sushi Restaurant,Japanese Restaurant,Food Stand,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Dumpling Restaurant
12,Windsor Terrace,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Fish Market,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar
31,Manhattan Beach,Sushi Restaurant,Japanese Restaurant,Poke Place,Fish Market,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar
32,Coney Island,Sushi Restaurant,Vietnamese Restaurant,Food Stand,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Dumpling Restaurant
36,Gerritsen Beach,Sushi Restaurant,Vietnamese Restaurant,Food Stand,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Dumpling Restaurant
45,Bergen Beach,Sushi Restaurant,Japanese Restaurant,Food Stand,BBQ Joint,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Dumpling Restaurant
